The covariance matrix is given by the matrix inversion lemma (also known as the Woodbury matrix identity). 

The expression is:

$ C_{\theta | Y} = (C_{\theta\theta}^{-1} + H^T C_{VV}^{-1} H)^{-1} $

The covariance matrix in a Bayesian linear regression context.

In [1]:
using FFTW

In [2]:
include("operations.jl");

In [3]:
using LinearAlgebra
using Distributions: cov, vec

# Simulated data
Y = [1.0, 2.0, 3.0]
H = [1.0; 2.0; 3.0]

# Prior covariance matrix (assuming it's a diagonal matrix for simplicity)
c₀₀ = Diagonal([1.0])

# Likelihood (assuming Gaussian errors)
cᵥᵥ = 0.1 * I

UniformScaling{Float64}
0.1*I

In [4]:
# Bayesian update to calculate posterior covariance
c₀ₗᵥ = ((c₀₀)⁻¹ .+ (H)ᵀ *  (cᵥᵥ)⁻¹ .* H)⁻¹

3×3 Matrix{Float64}:
  1.59192e14  -3.18384e14   1.59192e14
 -3.18384e14   6.36769e14  -3.18384e14
  1.59192e14  -3.18384e14   1.59192e14

In [5]:
using Plots

# Plotting the covariance matrix as a heatmap
plot1 = heatmap(c₀ₗᵥ, xlabel="Variables", ylabel="Variables", title="Covariance Matrix", c=:viridis)
savefig(plot1,"images/plot1.png");

<img src="images/plot1.png" width='' height='' > </img>

### Covariance Numerical Example

In [6]:
Supertech = [-.2 .1 .3 .5] 
Slowpoke = [.05 .2 -.12 .09] # space separator means Row Vector

1×4 Matrix{Float64}:
 0.05  0.2  -0.12  0.09

In [7]:
vec(Slowpoke) # use vec to convert to Column Vector

4-element Vector{Float64}:
  0.05
  0.2
 -0.12
  0.09

In [8]:
# Excel is using the "uncorrected" covariance
answer = -0.004875 # Scalar

-0.004875

In [9]:
answer == cov(vec(Supertech), vec(Slowpoke))

false

In [10]:
answer == cov(vec(Supertech), vec(Slowpoke), corrected=false)

true

In [11]:
H' == transpose.(H) 

false

In [12]:
H == transpose.(H) # Transpose element-wise doesn't do a thing bug ??

true

In [13]:
transpose(H) == permutedims(H) == H'

true

### Toeplitz Matrix Example

In [14]:
using Plots

In [15]:
# Function to generate a Toeplitz matrix
function toeplitz_matrix(c)
    n = length(c)
    T = zeros(n, n)
    for i in 1:n
        for j in 1:n
            T[i, j] = c[abs(i - j) + 1]
        end
    end
    return T
end;

In [16]:
# Example: Toeplitz matrix with first column [1, 2, 3, 4]
c = [1, 2, 3, 4]
T₀ = toeplitz_matrix(c)

4×4 Matrix{Float64}:
 1.0  2.0  3.0  4.0
 2.0  1.0  2.0  3.0
 3.0  2.0  1.0  2.0
 4.0  3.0  2.0  1.0

In [17]:
# Plot the Toeplitz matrix using heatmap
plot2 = 
heatmap(T₀, xlabel="Column Index", ylabel="Row Index", title="Toeplitz Matrix", c=:viridis)
savefig(plot2, "images/plot2.png");

<img src="images/plot2.png" width="" height="" > </img>

In [18]:
using ToeplitzMatrices

In [19]:
# Create a Toeplitz matrix using the toeplitz function
T = Toeplitz(1:length(c),c)

4×4 Toeplitz{Int64, UnitRange{Int64}, Vector{Int64}}:
 1  2  3  4
 2  1  2  3
 3  2  1  2
 4  3  2  1

In [20]:
# Plot the Toeplitz matrix using heatmap
plot3 = heatmap(T, xlabel="Column Index", ylabel="Row Index", title="Toeplitz Matrix", c=:viridis)
savefig(plot3, "images/plot3.png");

<img src="images/plot2.png" width="" height="" > </img>

# References
- [ ] [How to create simple covariance in Julia on a matrix](https://stackoverflow.com/questions/41879024/how-to-create-simple-covariance-in-julia-on-a-matrix)
- [ ] [Bessel's correction](https://en.wikipedia.org/wiki/Bessel%27s_correction)
> Bessel's correction is the use of n − 1 instead of n in the formula for the sample variance and sample standard deviation,

In [21]:
using ToeplitzMatrices

# Create two random Toeplitz matrices
n = 4
# a = rand(1:5, n)
# b = rand(1:5, n)
# A = Toeplitz(1:length(a),a)
# B = Toeplitz(1:length(b),b)

# Multiply the Toeplitz matrices
C = T₀ * T

# Check if the result is Toeplitz
is_toeplitz = issymmetric(C) && all(diag(C, k) == diag(C, k)[1] for k in -n+1:n-1)

println("Matrix T₀:")
println(T₀)

println("\nMatrix T:")
println(T)

println("\nMatrix C (Product of A and B):")
println(C)

println("\nIs C Toeplitz? ", is_toeplitz)


Matrix T₀:
[1.0 2.0 3.0 4.0; 2.0 1.0 2.0 3.0; 3.0 2.0 1.0 2.0; 4.0 3.0 2.0 1.0]

Matrix T:
[1 2 3 4; 2 1 2 3; 3 2 1 2; 4 3 2 1]

Matrix C (Product of A and B):
[30.0 22.0 18.0 20.0; 22.0 18.0 16.0 18.0; 18.0 16.0 18.0 22.0; 20.0 18.0 22.0 30.0]

Is C Toeplitz? false


In [22]:
x₁, x₂, x₃ = 1, 2, 3

(1, 2, 3)

In [23]:
Xₓ = [ x₁, x₂, x₃ ]

3-element Vector{Int64}:
 1
 2
 3

In [24]:
# Create a complex matrix
A = [1.0 + 2.0im  3.0 - 1.0im
     4.0 + 5.0im  6.0 - 4.0im]

# Calculate the complex conjugate matrix using the ' operator
adjoint(A) == A'


true

In [25]:
conj(A') == conj(adjoint(A))

true

In [26]:
conj(adjoint(A)) == adjoint(conj(A))

true

In [27]:
adjoint(conj(A))

2×2 adjoint(::Matrix{ComplexF64}) with eltype ComplexF64:
 1.0+2.0im  4.0+5.0im
 3.0-1.0im  6.0-4.0im

In [28]:
H = transpose(conj(A))

2×2 transpose(::Matrix{ComplexF64}) with eltype ComplexF64:
 1.0-2.0im  4.0-5.0im
 3.0+1.0im  6.0+4.0im

In [29]:
adjoint(A)

2×2 adjoint(::Matrix{ComplexF64}) with eltype ComplexF64:
 1.0-2.0im  4.0-5.0im
 3.0+1.0im  6.0+4.0im

In [30]:
transpose(A)

2×2 transpose(::Matrix{ComplexF64}) with eltype ComplexF64:
 1.0+2.0im  4.0+5.0im
 3.0-1.0im  6.0-4.0im

In [31]:
conj(A)

2×2 Matrix{ComplexF64}:
 1.0-2.0im  3.0+1.0im
 4.0-5.0im  6.0+4.0im

In [32]:
A

2×2 Matrix{ComplexF64}:
 1.0+2.0im  3.0-1.0im
 4.0+5.0im  6.0-4.0im

In [33]:
z1 = A
z2 = conj(A)

2×2 Matrix{ComplexF64}:
 1.0-2.0im  3.0+1.0im
 4.0-5.0im  6.0+4.0im

In [34]:
A[1]

1.0 + 2.0im

In [42]:
using Plots

# Define the complex matrix A
A = [1.0 + 2.0im  3.0 - 1.0im
     4.0 + 5.0im  6.0 - 4.0im]

# Plot the complex numbers in the complex plane
plot4 =
scatter(real(A)
    , imag(A)
    , markersize=8
    , label="Matrix A"
    , xlabel="Real"
    , ylabel="Imaginary"
    , legend=:topleft
    , ratio=1
)
# Plot the complex conjugate in the complex plane
plot4 = 
scatter!(real((A)ᴴ)
    , imag((A)ᴴ)
    , label="Matrix Aᴴ"
    , markersize=20

)
savefig(plot4,"images/plot4.png");

<img src="images/plot4.png" width='' height='' ></img>

In [43]:
# Define a square complex-valued matrix
B = [1.0 + 2.0im  3.0 - 1.0im  5.0 + 0.5im
     4.0 + 5.0im  6.0 - 4.0im  2.0 - 3.0im
     0.0 - 1.5im  2.0 + 3.0im  7.0 + 1.0im]

# Display the matrix
println("Complex-Valued Matrix B:")
println(B)


Complex-Valued Matrix B:
ComplexF64[1.0 + 2.0im 3.0 - 1.0im 5.0 + 0.5im; 4.0 + 5.0im 6.0 - 4.0im 2.0 - 3.0im; 0.0 - 1.5im 2.0 + 3.0im 7.0 + 1.0im]


In [44]:
B == (B)ᴴ

false

In [45]:
B == conj(transpose(B))

false

In [46]:
# Define a complex-valued matrix
C = [1.0 + 2.0im  3.0 - 1.0im
     4.0 + 5.0im  6.0 - 4.0im]

# Display the original complex-valued matrix
println("Complex-Valued Matrix C:")
println(C)

# Calculate and display the transpose of the matrix
C_transpose = transpose(C)
println("\nTranspose of Complex-Valued Matrix C:")
println(C_transpose)

# Calculate and display the complex conjugate of the transposed matrix
C_conjugate_transpose = conj(C_transpose)
println("\nComplex Conjugate of Transposed Matrix:")
println(C_conjugate_transpose)


Complex-Valued Matrix C:
ComplexF64[1.0 + 2.0im 3.0 - 1.0im; 4.0 + 5.0im 6.0 - 4.0im]

Transpose of Complex-Valued Matrix C:
ComplexF64[1.0 + 2.0im 4.0 + 5.0im; 3.0 - 1.0im 6.0 - 4.0im]

Complex Conjugate of Transposed Matrix:
ComplexF64[1.0 - 2.0im 4.0 - 5.0im; 3.0 + 1.0im 6.0 + 4.0im]


In [48]:
# Plot the complex numbers in the complex plane
plot5 = scatter(real(C)
    , imag(C)
    , markersize=8
    , label="Matrix C"
    , xlabel="Real"
    , ylabel="Imaginary"
    , legend=:topleft
    , ratio=1
)
# Plot the complex conjugate in the complex plane
plot5 = scatter!(real(C_conjugate_transpose)
    , imag(C_conjugate_transpose)
    , label="Matrix C_conjugate_transposeᴴ"
    , markersize=20

)
savefig(plot5,"images/plot5.png");

<img src="images/plot5.png" width='' height='' ></img>

In [49]:
A == C

true

In [50]:
(A)ᴴ == C_conjugate_transpose

true